### QED-C Application-Oriented Benchmarks - Hamiltonian Simulation with HamLib - Observables (Customizable)

The notebook contains specific examples for the HamLib-based Hamiltonian Simulation benchmark program.
Configure and run the cell below with the desired execution settings.
Then configure and run the remaining cell(s), each one a variation of this benchmark.

Note: This set of benchmarks surfaces the series of second-level functions used to benchmark HamLib observables.
This is a WORK-IN-PROGRESS and is provided to enable experimentation with alternative techniques for computing observables.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from math import sin, cos, pi
import time

# Configure module paths
import sys
sys.path.insert(1, "_common")
sys.path.insert(1, "qiskit")

# Import HamLib helper functions (from _common)
import hamlib_utils

# Import Hamlib Simulation kernel (from qiskit)
import hamlib_simulation_kernel

# Import Observable helper functions
import observables
import evolution_exact

verbose = True

#### for executing circuits to compute observables ...

# Import Qiskit and Qiskit Pauli operator classes
#from qiskit.quantum_info import Pauli, SparsePauliOp
#from qiskit import QuantumCircuit

# Initialize simulator backend
#from qiskit_aer import Aer
#backend = Aer.get_backend('qasm_simulator')


### Configure Options for Execution

In [3]:
# Qubit width of the Hamiltonian
num_qubits = 16

# Number of shots used for execution
num_shots = 10000

# Parameters of Trotterized simulation
K = 1
t = 0.1

# Define initial state; "checkerboard", "neele", "ghz", or a string consisting of 0s and 1s
init_state = "checkerboard"

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False

# arrange Hamiltonian terms into groups as specified by group method
# ... can be None, "simple", "1", "2", or "N" where N is < num_qubits
group_method = "simple"

# option to use new method for producing diagonalized measurement circuts with N-qubit groups
use_diag_method = False

# for testing the new method
#group_method = "N"
#use_diag_method = True


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [4]:
# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'

hamiltonian_name = 'TFIM'

print(f"... Hamiltonian name = {hamiltonian_name}")

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }

if hamiltonian_name == 'Heisenberg':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "pbc", "enc": "bk", "U":12 }

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "nonpbc", "enc": "gray", "U":10 }

if hamiltonian_name == 'Max3Sat':
    hamiltonian_params = { "ratio": "2", "rinst": "02" }

if hamiltonian_name == 'chemistry/electronic/standard/H2':
    hamiltonian_params = { "ham_BK": '' }

# load the HamLib file for the given hamiltonian name
hamlib_utils.load_hamlib_file(filename=hamiltonian_name)

# return a sparse Pauli list of terms queried from the open HamLib file
sparse_pauli_terms, dataset_name = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
print(f"... dataset_name = {dataset_name}")
print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")

print("")


... Hamiltonian name = TFIM
... dataset_name = graph-1D-grid-pbc-qubitnodes_Lx-16_h-2
... sparse_pauli_terms = 
[({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 15: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({15: 'Z', 14: 'Z'}, (1+0j)), ({2: 'Z', 3: 'Z'}, (1+0j)), ({3: 'Z', 4: 'Z'}, (1+0j)), ({4: 'Z', 5: 'Z'}, (1+0j)), ({5: 'Z', 6: 'Z'}, (1+0j)), ({6: 'Z', 7: 'Z'}, (1+0j)), ({7: 'Z', 8: 'Z'}, (1+0j)), ({8: 'Z', 9: 'Z'}, (1+0j)), ({9: 'Z', 10: 'Z'}, (1+0j)), ({10: 'Z', 11: 'Z'}, (1+0j)), ({11: 'Z', 12: 'Z'}, (1+0j)), ({12: 'Z', 13: 'Z'}, (1+0j)), ({13: 'Z', 14: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({15: 'X'}, (2+0j)), ({2: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({4: 'X'}, (2+0j)), ({5: 'X'}, (2+0j)), ({6: 'X'}, (2+0j)), ({7: 'X'}, (2+0j)), ({8: 'X'}, (2+0j)), ({9: 'X'}, (2+0j)), ({10: 'X'}, (2+0j)), ({11: 'X'}, (2+0j)), ({12: 'X'}, (2+0j)), ({13: 'X'}, (2+0j)), ({14: 'X'}, (2+0j))]



### Group the Pauli Terms and Generate Merged Terms
Here, we optimize the grouping of the Pauli terms, both for creating the base circuit with evolution terms and for generating the measurement circuits.

In [5]:
print(f"... using group method: {group_method}")

ts = time.time()

# use no grouping or the most basic method "simple"
if group_method == None or group_method == "simple":

    # Flag to control optimize by use of commuting groups
    use_commuting_groups = False
    if group_method == "simple":
        use_commuting_groups = True

    # group Pauli terms for quantum execution, optionally combining commuting terms into groups.
    pauli_term_groups, pauli_str_list = observables.group_pauli_terms_for_execution(
            num_qubits, sparse_pauli_terms, use_commuting_groups)

# use k-commuting algorithm
else:
    from generate_pauli_groups import compute_groups
    pauli_term_groups = compute_groups(num_qubits, sparse_pauli_terms, 1)

print(f"\n... Number of groups created: {len(pauli_term_groups)}")
print(f"... Pauli Term Groups:")
for group in pauli_term_groups:
    print(group)

group_time = round(time.time()-ts, 3)
print(f"\n... finished grouping terms, total grouping time = {group_time} sec.\n")

# for each group, create a merged pauli string from all the terms in the group
# DEVNOTE: move these 4 lines to a function in observables
pauli_str_list = []
for group in pauli_term_groups:
    merged_pauli_str = observables.merge_pauli_terms(group, num_qubits)
    pauli_str_list.append(merged_pauli_str)

print(f"\n... Merged Pauli strings, one per group:\n  {pauli_str_list}\n")


... using group method: simple

... Number of groups created: 2
... Pauli Term Groups:
[('ZZIIIIIIIIIIIIII', (1+0j)), ('ZIIIIIIIIIIIIIIZ', (1+0j)), ('IZZIIIIIIIIIIIII', (1+0j)), ('IIIIIIIIIIIIIIZZ', (1+0j)), ('IIZZIIIIIIIIIIII', (1+0j)), ('IIIZZIIIIIIIIIII', (1+0j)), ('IIIIZZIIIIIIIIII', (1+0j)), ('IIIIIZZIIIIIIIII', (1+0j)), ('IIIIIIZZIIIIIIII', (1+0j)), ('IIIIIIIZZIIIIIII', (1+0j)), ('IIIIIIIIZZIIIIII', (1+0j)), ('IIIIIIIIIZZIIIII', (1+0j)), ('IIIIIIIIIIZZIIII', (1+0j)), ('IIIIIIIIIIIZZIII', (1+0j)), ('IIIIIIIIIIIIZZII', (1+0j)), ('IIIIIIIIIIIIIZZI', (1+0j))]
[('XIIIIIIIIIIIIIII', (2+0j)), ('IXIIIIIIIIIIIIII', (2+0j)), ('IIIIIIIIIIIIIIIX', (2+0j)), ('IIXIIIIIIIIIIIII', (2+0j)), ('IIIXIIIIIIIIIIII', (2+0j)), ('IIIIXIIIIIIIIIII', (2+0j)), ('IIIIIXIIIIIIIIII', (2+0j)), ('IIIIIIXIIIIIIIII', (2+0j)), ('IIIIIIIXIIIIIIII', (2+0j)), ('IIIIIIIIXIIIIIII', (2+0j)), ('IIIIIIIIIXIIIIII', (2+0j)), ('IIIIIIIIIIXIIIII', (2+0j)), ('IIIIIIIIIIIXIIII', (2+0j)), ('IIIIIIIIIIIIXIII', (2+0j)), ('IIIIIIIII

### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

Optionally, create the base circuit using grouped terms (TBD).

In [6]:
# create Trotterized evolution circuit for HamLib Hamiltonian
ts = time.time()
qc, _ = hamlib_simulation_kernel.HamiltonianSimulation(
    num_qubits=num_qubits, 
    ham_op=sparse_pauli_terms,
    K=K, t=t,
    init_state = init_state,
    append_measurements = False,
    method = 1, 
)

create_time = round(time.time()-ts, 3)
print(f"\n... finished creating base circuit, total creation time = {create_time} sec.\n")

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 11:
    if hamiltonian_name != '':
        hamlib_simulation_kernel.kernel_draw(qc, method=1)
    else:
        print(qc)
else:
    print("  ... circuit is too large to draw.")

print("")



... finished creating base circuit, total creation time = 0.013 sec.

... Trotterized Circuit, K=1, t=0.1
  ... circuit is too large to draw.



### Create Measurement Circuits from Base Circuit and Pauli Terms
Here, we append basis rotation gates for each Pauli Term group to the base evolution circuit to create an array of circuits for execution.

In [7]:
# generate an array of circuits, one for each pauli_string in list
ts = time.time()

# use normal basis rotation and measurement circuits
if not use_diag_method:
    circuits = hamlib_simulation_kernel.create_circuits_for_pauli_terms(qc, num_qubits, pauli_str_list)

# option to use new method for producing diagonalized measurement circuts with N-qubit groups
else:
    print(f"... ****** using diagonalization method for measurement circuits")
    
    # generate an array of circuits, one for each pauli_string in list
    from generate_measurement_circuits import create_circuits_for_pauli_terms
    
    k = 2         # what is the k variable?
    
    circuits = [create_circuits_for_pauli_terms(qc, ops, 2) for ops in pauli_term_groups]

print(f"... Appended {len(circuits)} circuits, one for each group:")               
for circuit, group in list(zip(circuits, pauli_term_groups)):
    print(group)
    #print(circuit)

append_time = round(time.time()-ts, 3)
print(f"\n... finished appending {len(circuits)} measurement circuits, total creating time = {append_time} sec.\n")


... Appended 2 circuits, one for each group:
[('ZZIIIIIIIIIIIIII', (1+0j)), ('ZIIIIIIIIIIIIIIZ', (1+0j)), ('IZZIIIIIIIIIIIII', (1+0j)), ('IIIIIIIIIIIIIIZZ', (1+0j)), ('IIZZIIIIIIIIIIII', (1+0j)), ('IIIZZIIIIIIIIIII', (1+0j)), ('IIIIZZIIIIIIIIII', (1+0j)), ('IIIIIZZIIIIIIIII', (1+0j)), ('IIIIIIZZIIIIIIII', (1+0j)), ('IIIIIIIZZIIIIIII', (1+0j)), ('IIIIIIIIZZIIIIII', (1+0j)), ('IIIIIIIIIZZIIIII', (1+0j)), ('IIIIIIIIIIZZIIII', (1+0j)), ('IIIIIIIIIIIZZIII', (1+0j)), ('IIIIIIIIIIIIZZII', (1+0j)), ('IIIIIIIIIIIIIZZI', (1+0j))]
[('XIIIIIIIIIIIIIII', (2+0j)), ('IXIIIIIIIIIIIIII', (2+0j)), ('IIIIIIIIIIIIIIIX', (2+0j)), ('IIXIIIIIIIIIIIII', (2+0j)), ('IIIXIIIIIIIIIIII', (2+0j)), ('IIIIXIIIIIIIIIII', (2+0j)), ('IIIIIXIIIIIIIIII', (2+0j)), ('IIIIIIXIIIIIIIII', (2+0j)), ('IIIIIIIXIIIIIIII', (2+0j)), ('IIIIIIIIXIIIIIII', (2+0j)), ('IIIIIIIIIXIIIIII', (2+0j)), ('IIIIIIIIIIXIIIII', (2+0j)), ('IIIIIIIIIIIXIIII', (2+0j)), ('IIIIIIIIIIIIXIII', (2+0j)), ('IIIIIIIIIIIIIXII', (2+0j)), ('IIIIIIIIIIIIIIXI', (2

### Execute Circuits to Produce Measurement Distributions
For now, we execute using the Aer simulator. This will be enhanced to use the QED-C execution module to enable execution on multiple programming APIs and backend systems. 

In [8]:
# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

print(f"... begin executing {len(circuits)} circuits ...")
ts = time.time()

# Execute all of the circuits to obtain array of result objects
results = backend.run(circuits, num_shots=num_shots).result()

exec_time = round(time.time()-ts, 3)
print(f"... finished executing {len(circuits)} circuits, total execution time = {exec_time} sec.\n")


... begin executing 2 circuits ...
... finished executing 2 circuits, total execution time = 0.016 sec.



### Compute the Energy of the Evolved State from the Hamiltonian and the Measurement Distributions
Here, we process the measurement results from circuit execution using the Hamiltonian terms to compute the expectation value of the energy observable.

We also, retain the contributions from each terms so that we can use those in calcuating other observables from the same measurement results.

In [9]:
# Compute the total energy for the Hamiltonian

print(f"... begin computing observable value ...")
ts = time.time()

total_energy, term_contributions = observables.calculate_expectation_from_measurements(
                                            num_qubits, results, pauli_term_groups)
obs_time = round(time.time()-ts, 3)
print(f"... finished computing observable value, computation time = {obs_time} sec.\n")

print(f"    Total Energy: {round(np.real(total_energy), 4)}\n")
print(f"    Term Contributions: {term_contributions}\n")

total_time = group_time + create_time + append_time + exec_time + obs_time
total_time = round(total_time, 3)
print(f"\n... total observable computation time = {total_time} sec.\n")


... begin computing observable value ...
... finished computing observable value, computation time = 0.014 sec.

    Total Energy: -12.9805

    Term Contributions: {'ZZIIIIIIIIIIIIII': -0.859375, 'ZIIIIIIIIIIIIIIZ': -0.857421875, 'IZZIIIIIIIIIIIII': -0.861328125, 'IIIIIIIIIIIIIIZZ': -0.845703125, 'IIZZIIIIIIIIIIII': -0.861328125, 'IIIZZIIIIIIIIIII': -0.8515625, 'IIIIZZIIIIIIIIII': -0.8203125, 'IIIIIZZIIIIIIIII': -0.82421875, 'IIIIIIZZIIIIIIII': -0.859375, 'IIIIIIIZZIIIIIII': -0.85546875, 'IIIIIIIIZZIIIIII': -0.830078125, 'IIIIIIIIIZZIIIII': -0.8515625, 'IIIIIIIIIIZZIIII': -0.861328125, 'IIIIIIIIIIIZZIII': -0.822265625, 'IIIIIIIIIIIIZZII': -0.830078125, 'IIIIIIIIIIIIIZZI': -0.84375, 'XIIIIIIIIIIIIIII': 0.083984375, 'IXIIIIIIIIIIIIII': -0.015625, 'IIIIIIIIIIIIIIIX': 0.00390625, 'IIXIIIIIIIIIIIII': 0.013671875, 'IIIXIIIIIIIIIIII': 0.048828125, 'IIIIXIIIIIIIIIII': 0.0390625, 'IIIIIXIIIIIIIIII': 0.0546875, 'IIIIIIXIIIIIIIII': 0.0078125, 'IIIIIIIXIIIIIIII': 0.00390625, 'IIIIIIIIXIIIIIII': -

### Compute Energy of the Evolved State Classically and Compare to Quantum Result
Here, we use a classical computation to determine the exact expectation value and produce a quality score for the quantum computation.

In [10]:
print(f"... begin classical computation of expectation value ...")
                    
ts = time.time()

if num_qubits <= 20:
    correct_exp, correct_dist = evolution_exact.compute_expectation_exact(
            init_state,
            observables.ensure_pauli_terms(sparse_pauli_terms, num_qubits),
            1.0            # time
            )
else:
    correct_exp = 0.001
    correct_dist = None
    
exact_time = round(time.time()-ts, 3)
print(f"... exact computation time = {exact_time} sec")

print(f"\nExact expectation value, computed classically: {round(np.real(correct_exp), 4)}")
print(f"Estimated expectation value, computed using quantum algorithm: {round(np.real(total_energy), 4)}\n")

simulation_quality = round(np.real(total_energy) / np.real(correct_exp), 3)
print(f"    ==> Simulation Quality: {np.real(simulation_quality)}\n")



... begin classical computation of expectation value ...
... exact computation time = 0.793 sec

Exact expectation value, computed classically: -16.0
Estimated expectation value, computed using quantum algorithm: -12.9805

    ==> Simulation Quality: 0.811



### Compute Expectation with Estimator
Here, we do a sanity check on the above processes by comparing with energy obtained using the Qiskit Estimator.

In [11]:
# Call the Qiskit Estimator using the function wrapper in 'observables' module.

# DEVNOTE: We may want to surface the actual Estimator call instead. 

# Ensure that the pauli_terms are in 'full' format, not 'sparse' - convert if necessary
pauli_terms = observables.ensure_pauli_terms(sparse_pauli_terms, num_qubits=num_qubits)
pauli_terms = observables.swap_pauli_list(pauli_terms)

ts = time.time()

estimator_energy = observables.estimate_expectation_with_estimator(backend, qc, pauli_terms, num_shots=num_shots)

estimator_time = round(time.time()-ts, 3)
print(f"... Estimator computation time = {estimator_time} sec")

print(f"Expectation value, computed using Qiskit Estimator: {round(np.real(estimator_energy), 4)}\n")


... Estimator computation time = 0.519 sec
Expectation value, computed using Qiskit Estimator: -13.293



### Compute expectation value of other observables.
WORK-IN-PROGRESS: This code attempts to derive spin correlation and magenetization Hamiltonians from the Hamiltonian under test.
We do not currently check whether the terms actually make sense.  We also do not consider other observable types at this point.

In [12]:

print(f"Hamiltonian under test:")
print(pauli_term_groups)
print("")

################
### DEVNOTE: We don't want to use SparsePauliOp if possible

from qiskit.quantum_info import SparsePauliOp

# need to check if the below terms are actually in the Hamiltonian

L = num_qubits

# this doesn't include the Z's at the ends of the chain; OK, if we assume an open chain
print("****")
correlation_op = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, i + 1], 1.0) for i in range(0, L - 1)],
    num_qubits=L
) / (L - 1)
H_terms_spin_correlation = correlation_op.to_list()
print("... mean spin correlation terms: ", H_terms_spin_correlation)

print("****")
magnetization_op = SparsePauliOp.from_sparse_list(
    #[("Z", [i], 1.0) for i in range(0, 6)], num_qubits=6
    [("Z", [i], 1.0) for i in range(0, L)], num_qubits=L
)
H_terms_magnetization = magnetization_op.to_list()
print("... magnetization terms: ", H_terms_magnetization)

print("")

spin_correlation = observables.calculate_expectation_from_contributions(term_contributions, H_terms_spin_correlation)
print(f"Spin Correlation: {round(np.real(spin_correlation), 3)}")

magnetization = observables.calculate_expectation_from_contributions(term_contributions, H_terms_magnetization)
print(f"Magnetization: {round(np.real(magnetization), 3)}")

print("")


Hamiltonian under test:
[[('ZZIIIIIIIIIIIIII', (1+0j)), ('ZIIIIIIIIIIIIIIZ', (1+0j)), ('IZZIIIIIIIIIIIII', (1+0j)), ('IIIIIIIIIIIIIIZZ', (1+0j)), ('IIZZIIIIIIIIIIII', (1+0j)), ('IIIZZIIIIIIIIIII', (1+0j)), ('IIIIZZIIIIIIIIII', (1+0j)), ('IIIIIZZIIIIIIIII', (1+0j)), ('IIIIIIZZIIIIIIII', (1+0j)), ('IIIIIIIZZIIIIIII', (1+0j)), ('IIIIIIIIZZIIIIII', (1+0j)), ('IIIIIIIIIZZIIIII', (1+0j)), ('IIIIIIIIIIZZIIII', (1+0j)), ('IIIIIIIIIIIZZIII', (1+0j)), ('IIIIIIIIIIIIZZII', (1+0j)), ('IIIIIIIIIIIIIZZI', (1+0j))], [('XIIIIIIIIIIIIIII', (2+0j)), ('IXIIIIIIIIIIIIII', (2+0j)), ('IIIIIIIIIIIIIIIX', (2+0j)), ('IIXIIIIIIIIIIIII', (2+0j)), ('IIIXIIIIIIIIIIII', (2+0j)), ('IIIIXIIIIIIIIIII', (2+0j)), ('IIIIIXIIIIIIIIII', (2+0j)), ('IIIIIIXIIIIIIIII', (2+0j)), ('IIIIIIIXIIIIIIII', (2+0j)), ('IIIIIIIIXIIIIIII', (2+0j)), ('IIIIIIIIIXIIIIII', (2+0j)), ('IIIIIIIIIIXIIIII', (2+0j)), ('IIIIIIIIIIIXIIII', (2+0j)), ('IIIIIIIIIIIIXIII', (2+0j)), ('IIIIIIIIIIIIIXII', (2+0j)), ('IIIIIIIIIIIIIIXI', (2+0j))]]

****
... m